In [5]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
import asammdf
Signal = asammdf.Signal
import numpy as np
import py
import get_files
import uuid
%matplotlib inline

In [7]:
%%file MDF2.py
import asammdf
def _path(self):
    return py.path.local(self._file.name)

def __repr__(self):
    return "MDF<{}, v={}>".format(self._path.basename, self.version)

# Dunctions to extend MDF with.
extend = {
    "__repr__": __repr__,
    "_path": property(_path),
}
MDF2 = type("MDF2", (asammdf.MDF, ), extend)

Overwriting MDF2.py


In [8]:
from MDF2 import MDF2

In [12]:
from signal_generators import *

In [16]:
import time

In [21]:
t0 = time.time()
T = np.zeros(0)

In [43]:
np.float32

numpy.float32

In [75]:
class SignalBuffer():
    def __init__(self, buffer_size=1024, dtype=np.float32):
        self.buffer_size = buffer_size
        self.index=0
        self.uuid=uuid.uuid4()
        self.timestamps = np.ones(
            self.buffer_size,
            dtype=dtype,
        )*np.NaN
        self.samples = np.ones(
            self.buffer_size,
            dtype=dtype,
        )*np.NaN
        
    def append(self, timestamp, sample):
        try:
            self.timestamps[self.index] = timestamp
            self.samples[self.index] = sample
            self.index+=1
        except:
            # TODO: Something
            raise
            
    def signal(self, name=None, unit=""):
        if name is None:
            name = self.uuid
        signal_ = asammdf.Signal(
            timestamps=self.timestamps[:self.index],
            samples=self.samples[:self.index],
            name=name,
            unit=unit,
            comment="buffer",
        )
        return signal_
            
    @property
    def _free(self):
        return self.buffer-self.index
        
    def __repr__(self):
        return "SignalBuffer<free={}>".format(self._free)

In [76]:
utc_buffer = SignalBuffer()
utc_buffer2 = SignalBuffer()

t0 = time.time()
for _ in range(10):
    utc = time.time()
    utc_buffer.append(
        timestamp=utc-t0,
        sample=utc,
    )
    time.sleep(0.1)
    
for _ in range(10):
    utc = time.time()
    utc_buffer2.append(
        timestamp=utc-t0,
        sample=utc,
    )
    time.sleep(0.1)

In [81]:
f = utc_buffer.signal(name="UTC", unit="s").extend(utc_buffer2.signal())

In [87]:
import datetime

In [94]:
T = datetime.datetime.utcnow()

In [103]:
T.strftime("%Y%M%d")

'20180219'

In [105]:
T.strftime("%m-%B")

'02-February'

In [107]:
T.strftime("%Y%m%d-Testing")

'20180219-Testing'

In [113]:
import os
import py.path

In [116]:
channel_path_ = [
    "Data",
    T.strftime("%Y"),
    T.strftime("%m-%B"),
    T.strftime("%Y%m%d-Testing"),
]

channel_path = py.path.local(
        os.path.join(*channel_path_)
)

channel_path.dirpath().ensure(dir=True)

local('/projects/RaspberryMDF/DevNotebooks/Data/2018/02-February')

In [118]:
channel_path.exists()

False